### nber experiments to get and explore feataure vectors
data from see
https://sites.google.com/site/patentdataproject/Home/downloads


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine
from matplotlib.colors import LogNorm

%matplotlib inline
%reload_ext line_profiler


In [2]:
!ls -l

total 2400952
-rw-r--r--  1 ahuxor  staff      21454 18 Jul 13:04 NBER_generality_originality_heatmap.pdf
-rw-r--r--  1 ahuxor  staff       1097 22 Jul 19:34 Untitled.ipynb
drwxr-xr-x  4 ahuxor  staff        136 23 Jul 14:59 Update_to_NBER
-rw-r--r--@ 1 ahuxor  staff   51973915  4 Aug  2010 assignee.dta
-rwxr-xr-x@ 1 ahuxor  staff  236509670 28 Mar  2009 cite76_06.dta
-rw-rw-rw-@ 1 ahuxor  staff     788072 28 Jul  2010 dynass.dat
-rw-r--r--  1 ahuxor  staff     750569 21 Jul 12:37 nber_experiments.ipynb
-rw-r--r--  1 ahuxor  staff     255320 22 Jul 22:12 nber_feature_vectors.ipynb
-rwxr-xr-x@ 1 ahuxor  staff   56791108 17 Aug  2013 orig_gen_76_06.dta
-rw-r--r--@ 1 ahuxor  staff  364030957  4 Aug  2010 pat76_06_assg.dta
-rw-r--r--@ 1 ahuxor  staff  252609858  4 Aug  2010 pat76_06_ipc.dta
-rw-r--r--@ 1 ahuxor  staff  263827184  7 Apr  2008 patassg.dta
-rw-r--r--@ 1 ahuxor  staff    1702925 19 Dec  2008 pdpcohdr.dat
drwxr-xr-x  7 ahuxor  staff        238  8 Jun 11:21 user_documentation


In [3]:
# MAIN
# read in main data file in the Strata9 .dta format
!date
pat76_06_assg_df = pd.read_stata('pat76_06_assg.dta')
!date

Mon 24 Jul 2017 11:53:55 BST
Mon 24 Jul 2017 11:55:26 BST


In [4]:
pat76_06_assg_df.head(6).T

,0,1,2,3,4,5
allcites,6,0,20,4,1,3
appyear,1974,1974,1975,1974,1975,1972
asscode,2,2,2,1,1,2
assgnum,1,1,1,NaN,NaN,1
cat,6,6,6,5,6,6
cat_ocl,6,6,6,NaN,6,6
cclass,2/161.4,5/11,5/430,114/354,12/142S,15/53.4
country,US,US,US,,,US
ddate,NaT,NaT,NaT,NaT,NaT,NaT
gday,6,6,6,6,6,6


In [4]:
# note that is some of the above, there are pdpass values which are NaN. These
# appear to be for lone inventor/assignee people. Remove these:
pat76_06_assg_df_valid_pdpass = pat76_06_assg_df[pat76_06_assg_df.pdpass.notnull()]

In [5]:
!date
unique_assignees = pat76_06_assg_df_valid_pdpass.pdpass.unique()
print(len(unique_assignees))
!date

Mon 24 Jul 2017 12:51:19 BST
223958
Mon 24 Jul 2017 12:51:19 BST


In [6]:
# read is assignee names datafile
# note that using dta file partly as the asc file has some formatting error asscoiated.
assignee_names = pd.read_stata('assignee.dta')



In [7]:
%%time
unique_icl_class = pat76_06_assg_df_valid_pdpass.icl_class.unique()
print(len(unique_icl_class))


3798
CPU times: user 293 ms, sys: 48 ms, total: 341 ms
Wall time: 350 ms


In [8]:
%%time
# now try and get vectors for each assignee of the number of patents they hols in each icl_class

# create final dataframe to hold results, starting with all classes as first column
feature_vectors_df = pd.DataFrame({'icl_class' : unique_icl_class})

# first get list of (unique) assignees (try first five first to test code)
test_range = unique_assignees[0:10]
# then for each one of them, 
for assignee in test_range:
    # look at all the patents they have
    temp = pat76_06_assg_df_valid_pdpass[pat76_06_assg_df_valid_pdpass.pdpass==assignee].icl_class
    icl_codes_in_current = temp.values
    #  and count instances of these
    n = len(unique_icl_class)
    icl_codes_per_assignee_vector = [0] * n 
    for m in range(0,n):
        current_icl_class = unique_icl_class[m]
        counted = list(icl_codes_in_current).count(current_icl_class)
        icl_codes_per_assignee_vector[m] = counted
        col_name_used = assignee.astype(str)
        feature_vectors_df[col_name_used]=icl_codes_per_assignee_vector


CPU times: user 34.2 s, sys: 792 ms, total: 35 s
Wall time: 41.5 s


In [114]:
# TRYING TO PROFILE
# now try and get vectors for each assignee of the number of patents they hols in each icl_class

# create final dataframe to hold results, starting with all classes as first column
def fv1(num):
    feature_vectors_df = pd.DataFrame({'icl_class' : unique_icl_class})

    # first get list of (unique) assignees (try first five first to test code)
    test_range = unique_assignees[0:num]
    # then for each one of them, 
    for assignee in test_range:
        # look at all the patents they have
        temp = pat76_06_assg_df_valid_pdpass[pat76_06_assg_df_valid_pdpass.pdpass==assignee].icl_class
        icl_codes_in_current = temp.values
        #  and count instances of these
        n = len(unique_icl_class)
        icl_codes_per_assignee_vector = [0] * n 
        for m in range(0,n):
            current_icl_class = unique_icl_class[m]
            counted = list(icl_codes_in_current).count(current_icl_class)
            icl_codes_per_assignee_vector[m] = counted
            col_name_used = assignee.astype(str)
            feature_vectors_df[col_name_used]=icl_codes_per_assignee_vector
    return

In [9]:
# TRYING TO PROFILE
# now try and get vectors for each assignee of the number of patents they hols in each icl_class

# create final dataframe to hold results, starting with all classes as first column
def fv2(num):
    feature_vectors_df = pd.DataFrame({'icl_class' : unique_icl_class})

    # first get list of (unique) assignees (try first five first to test code)
    test_range = unique_assignees[0:num]
    # then for each one of them, 
    for assignee in test_range:
        # look at all the patents they have
        temp = pat76_06_assg_df_valid_pdpass[pat76_06_assg_df_valid_pdpass.pdpass==assignee].icl_class
        icl_codes_in_current = temp.values
        #  and count instances of these
        n = len(unique_icl_class)
        icl_codes_per_assignee_vector = [0] * n 
        for m in range(0,n):
            current_icl_class = unique_icl_class[m]
            counted = list(icl_codes_in_current).count(current_icl_class)
            icl_codes_per_assignee_vector[m] = counted
            col_name_used = assignee.astype(str)
            # create new df from icl_codes_per_assignee_vector
            temp_df = pd.DataFrame(icl_codes_per_assignee_vector)
            temp_df.columns = [col_name_used]
            feature_vectors_df = feature_vectors_df#.join(temp_df)
    return

In [22]:
# create list for unique_icl_class, rather than numpy array
unique_icl_class_list = unique_icl_class.tolist()

In [108]:
# TRYING TO PROFILE
# TRY TO ONLY USE LISTS, TO THE VERY END
# now try and get vectors for each assignee of the number of patents they hols in each icl_class

# create final dataframe to hold results, starting with all classes as first column
# num = number of patents to look at
def fv3(num):
    # feature_vectors_df = pd.DataFrame({'icl_class' : unique_icl_class})
    feature_vectors_listoflist =[]
    # first get list of (unique) assignees (try first five first to test code)
    range_of_assignees = unique_assignees[0:num]
    # then for each one of them, 
    for assignee in range_of_assignees:
        # look at all the patent classes each assignee has
        temp = pat76_06_assg_df_valid_pdpass[pat76_06_assg_df_valid_pdpass.pdpass==assignee].icl_class
        icl_codes_in_current_list = list(temp.values)
        #  and count instances of these if multiple instances of any 
        n = len(unique_icl_class)
        icl_codes_per_assignee_list = []
        for m in range(0,n):
            current_icl_class = unique_icl_class_list[m]
            counted = icl_codes_in_current_list.count(current_icl_class)
            icl_codes_per_assignee_list.append(counted)
            col_name_used = assignee.astype(str)
        #feature_vectors_df[col_name_used]=icl_codes_per_assignee_vector
        feature_vectors_listoflist.append((icl_codes_per_assignee_list))
    return(feature_vectors_listoflist)

In [118]:
j=10
%lprun -f fv3 xx= fv3(j)
test_out = fv3(j)

In [125]:
%%time
j=10000
test_out = fv3(j)

CPU times: user 5min 30s, sys: 4.59 s, total: 5min 34s
Wall time: 5min 41s


In [119]:
print('length test_out = ', len(test_out))
inside=test_out[0]
print('length x = ', len(inside))



length test_out =  10
length x =  3798


In [120]:
%%time
test_out_df = pd.DataFrame(test_out)

CPU times: user 356 ms, sys: 525 ms, total: 881 ms
Wall time: 953 ms


In [122]:
test_out_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,3788,3789,3790,3791,3792,3793,3794,3795,3796,3797
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
unique_icl_class_list[0]

'A41D'

In [56]:
junk =[[1,2,3],[4,5,6]]
print(len(junk))
len(junk[0])

2


3

In [14]:
type(what)
len(what)

7596

In [14]:
temp_df = pd.DataFrame(icl_codes_per_assignee_vector)
temp_df.columns = ['test']
temp_df
test1 = feature_vectors_df.join(temp_df)
test1

,icl_class,10030271.0,10156902.0,10112031.0,10030276.0,10172187.0,10679134.0,10030281.0,10577942.0,10036908.0,...,10140712.0,10286499.0,10679547.0,10699376.0,10078381.0,10116963.0,10235236.0,10198604.0,10966684.0,test
0,A41D,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A47D,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,B60S,0,0,0,2,0,0,0,0,0,...,0,1,1,6,0,0,0,0,0,0
3,E01F,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A47L,0,0,0,0,9,1,1,0,0,...,0,4,0,1,0,0,0,0,0,0
5,A22C,0,0,0,0,0,0,0,10,0,...,0,0,0,0,0,0,0,0,0,0
6,A22B,0,0,0,0,0,0,0,0,7,...,0,0,0,0,0,0,0,0,0,0
7,D01G,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,B67B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,F16G,0,0,0,0,0,0,0,0,0,...,0,65,21,0,0,0,0,0,2,2


In [15]:
assignee_array = feature_vectors_df.columns.values
assignee_array[1]


'10030271.0'

In [16]:
# go through each column header and compare it to the first by cosine similarity
# NOTE: does not like NaN, so clear these out - if not useful

# set number of patents available
pnum = 100
# create a matrix to hold results
cos_ang_mat = np.zeros((pnum,pnum))
for i in range(1,pnum):
    tempi = assignee_array[i]
    for j in range(1,pnum):
        tempj = assignee_array[j]
        cos_similarity = 1-cosine(feature_vectors_df[tempi], feature_vectors_df[tempj])
        cos_ang_mat[i,j] = cos_similarity
        # print(cos_sim)
        if (cos_similarity>0.95) & (tempi!=tempj):
            print('close for ',tempi,tempj,' with cos_ang = ',cos_similarity)


close for  10030288.0 10030364.0  with cos_ang =  1.0
close for  10140808.0 11073260.0  with cos_ang =  0.971692299434
close for  10030332.0 10704074.0  with cos_ang =  0.983930955873
close for  10704074.0 10030332.0  with cos_ang =  0.983930955873
close for  10030357.0 10030383.0  with cos_ang =  1.0
close for  10030357.0 10526855.0  with cos_ang =  1.0
close for  10030357.0 10030387.0  with cos_ang =  1.0
close for  10030357.0 20030387.0  with cos_ang =  1.0
close for  10030364.0 10030288.0  with cos_ang =  1.0
close for  10030379.0 10030381.0  with cos_ang =  1.0
close for  10030379.0 10078950.0  with cos_ang =  0.9838699101
close for  10030381.0 10030379.0  with cos_ang =  1.0
close for  10030381.0 10078950.0  with cos_ang =  0.9838699101
close for  10078950.0 10030379.0  with cos_ang =  0.9838699101
close for  10078950.0 10030381.0  with cos_ang =  0.9838699101
close for  10030383.0 10030357.0  with cos_ang =  1.0
close for  10030383.0 10526855.0  with cos_ang =  1.0
close for  10

In [17]:
patent_ids = feature_vectors_df.columns.values[1:]
patent_ids

array(['10030271.0', '10156902.0', '10112031.0', '10030276.0',
       '10172187.0', '10679134.0', '10030281.0', '10577942.0',
       '10036908.0', '10085072.0', '10200651.0', '10030286.0',
       '10030288.0', '10048453.0', '10151655.0', '10680932.0',
       '11474328.0', '10515817.0', '10700759.0', '10105812.0',
       '10030298.0', '10204962.0', '10045633.0', '10030301.0',
       '10098338.0', '10131500.0', '10046713.0', '10155465.0',
       '10430930.0', '10157481.0', '10140808.0', '10285889.0',
       '10036902.0', '11105728.0', '10197599.0', '10045362.0',
       '10615562.0', '10446448.0', '10030327.0', '10030332.0',
       '10704074.0', '10030335.0', '10090805.0', '10207824.0',
       '10030338.0', '10120593.0', '10127546.0', '10031696.0',
       '10030344.0', '10078916.0', '10312446.0', '10030349.0',
       '10459323.0', '10052492.0', '10072465.0', '10031601.0',
       '10030354.0', '10461006.0', '10030357.0', '10107775.0',
       '10135673.0', '10634775.0', '10030364.0', '10256

In [18]:
feature_vectors_df.head()

,icl_class,10030271.0,10156902.0,10112031.0,10030276.0,10172187.0,10679134.0,10030281.0,10577942.0,10036908.0,...,10030414.0,10140712.0,10286499.0,10679547.0,10699376.0,10078381.0,10116963.0,10235236.0,10198604.0,10966684.0
0,A41D,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A47D,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,B60S,0,0,0,2,0,0,0,0,0,...,0,0,1,1,6,0,0,0,0,0
3,E01F,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A47L,0,0,0,0,9,1,1,0,0,...,0,0,4,0,1,0,0,0,0,0


In [19]:
# how many zeros in, say, the first patent column

yy = feature_vectors_df['10156902.0'].values
num_nonzeros = np.count_nonzero(yy)
num_zeros = len(yy) - num_nonzeros
print('length of array = ', len(yy))
print('number of zeros = ', num_zeros)
print('number of nonzeros [icl classes] = ', num_nonzeros)

length of array =  3798
number of zeros =  3776
number of nonzeros [icl classes] =  22


In [20]:
cos_ang_mat_df = pd.DataFrame(cos_ang_mat)
cos_ang_mat_df.columns = patent_ids
cos_ang_mat_df.set_index(patent_ids, inplace=True)
cos_ang_mat_df

,10030271.0,10156902.0,10112031.0,10030276.0,10172187.0,10679134.0,10030281.0,10577942.0,10036908.0,10085072.0,...,10030414.0,10140712.0,10286499.0,10679547.0,10699376.0,10078381.0,10116963.0,10235236.0,10198604.0,10966684.0
10030271.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10156902.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10112031.0,0.0,0.0,1.000000,0.045943,0.000000,0.012417,0.008977,0.000000,0.0,0.000000,...,0.089981,0.000000,0.000000,0.007019,0.010154,0.068645,0.007367,0.003090,0.005784,0.000000
10030276.0,0.0,0.0,0.045943,1.000000,0.000000,0.000000,0.011631,0.000000,0.0,0.000000,...,0.006178,0.000000,0.000000,0.036376,0.035081,0.019447,0.005113,0.000000,0.000000,0.000000
10172187.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.116445,...,0.018927,0.000000,0.000000,0.009584,0.016946,0.016497,0.000000,0.000000,0.000000,0.000000
10679134.0,0.0,0.0,0.012417,0.000000,0.000000,1.000000,0.070728,0.938315,0.0,0.000000,...,0.009898,0.000000,0.000000,0.042453,0.000000,0.003525,0.007831,0.000000,0.177211,0.000000
10030281.0,0.0,0.0,0.008977,0.011631,0.000000,0.070728,1.000000,0.075378,0.0,0.000000,...,0.001638,0.000000,0.000000,0.004039,0.015709,0.010195,0.002664,0.000000,0.000000,0.000000
10577942.0,0.0,0.0,0.000000,0.000000,0.000000,0.938315,0.075378,1.000000,0.0,0.000000,...,0.002288,0.000000,0.000000,0.042862,0.000000,0.003074,0.000000,0.000000,0.000000,0.000000
10036908.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10085072.0,0.0,0.0,0.000000,0.000000,0.116445,0.000000,0.000000,0.000000,0.0,1.000000,...,0.011616,0.000000,0.000000,0.008370,0.000000,0.001201,0.000863,0.000000,0.006323,0.000000


In [21]:
# these two assignees have a cos-ang, see above, of 0.938
print(assignee_names[assignee_names.pdpass==10172187])
print(assignee_names[assignee_names.pdpass==10030281])

                    cod    cod_fix    pdpass         standard_name  \
5156  02 US corporation  No change  10172187    ADVANCE MACHINE CO   
5157  02 US corporation  No change  10172187  ADVANCE MACHINE CORP   

      uspto_assignee  
5156          8665.0  
5157          8665.0  
                                     cod    cod_fix    pdpass standard_name  \
1392  03 Foreign corp, incl. state-owned  No change  10030281   A SUTTER AG   

      uspto_assignee  
1392         42390.0  


In [22]:
# these two assignees have a cos-ang, see above, of 0.972
print(assignee_names[assignee_names.pdpass==11073260])
print(assignee_names[assignee_names.pdpass==10140808])
# could these basically all be an array of zeros?!
# no - see cell below

                      cod    cod_fix    pdpass               standard_name  \
286855  02 US corporation  No change  11073260         THOMAS & BETTS CORP   
286856  02 US corporation  No change  11073260  THOMAS & BETTS CORPROATION   

        uspto_assignee  
286855        573320.0  
286856        573320.0  
                      cod    cod_fix    pdpass               standard_name  \
14912   02 US corporation  No change  10140808                         AMP   
14917   02 US corporation  No change  10140808                    AMP CORP   
14918   02 US corporation  No change  10140808              AMP CORPORATED   
14921   02 US corporation  No change  10140808      AMP GEN PATENT COUNSEL   
14922   02 US corporation  No change  10140808                     AMP INC   
14923   02 US corporation  No change  10140808             AMP INCORPORTED   
14924   02 US corporation  No change  10140808            AMP INCROPORATED   
107740  02 US corporation  No change  10140808  GEN PATENT COUNSEL

In [23]:
# these two assignees have a cos-ang, see above, of 0.9906
print(assignee_names[assignee_names.pdpass==10780383])
print('-----')
print(assignee_names[assignee_names.pdpass==10401313])


                      cod    cod_fix    pdpass               standard_name  \
212860  02 US corporation  No change  10780383    ORTHO PHAMACEUTICAL CORP   
212861  02 US corporation  No change  10780383                 ORTHO PHARM   
212863  02 US corporation  No change  10780383            ORTHO PHARM CORP   
212864  02 US corporation  No change  10780383        ORTHO PHARM CORP INC   
212867  02 US corporation  No change  10780383   ORTHO PHARMACEITICAL CORP   
212868  02 US corporation  No change  10780383  ORTHO PHARMACEUCTICAL CORP   

        uspto_assignee  
212860        421765.0  
212861        421765.0  
212863        421765.0  
212864        421765.0  
212867        421765.0  
212868        421765.0  
-----
                     cod    cod_fix    pdpass         standard_name  \
84581  02 US corporation  No change  10401313              ELI & CO   
84587  02 US corporation  No change  10401313        ELI LIILY & CO   
84588  02 US corporation  No change  10401313       ELI LIL

In [24]:
# these two assignees have a cos-ang, see above, of 0.9886
print(assignee_names[assignee_names.pdpass==10109257])
print(assignee_names[assignee_names.pdpass==10198901])


                     cod    cod_fix    pdpass                standard_name  \
41726  02 US corporation  No change  10109257          BUROUGHS WELCOME CO   
41762  02 US corporation  No change  10109257         BURROUGHS WELCOME CO   
41763  02 US corporation  No change  10109257           BURROUGHS WELLCOME   
41764  02 US corporation  No change  10109257  BURROUGHS WELLCOME & CO INC   
41765  02 US corporation  No change  10109257        BURROUGHS WELLCOME CO   
41766  02 US corporation  No change  10109257    BURROUGHS WELLCOME CO 141   
41767  02 US corporation  No change  10109257       BURROUGHS WELLCOME INC   
41768  02 US corporation  No change  10109257       BURROUGHTS WELLCOME CO   

       uspto_assignee  
41726         81605.0  
41762         81605.0  
41763         81605.0  
41764         81605.0  
41765         81605.0  
41766         81605.0  
41767         81605.0  
41768         81605.0  
                                      cod    cod_fix    pdpass  \
21948  03 Forei

In [25]:
# these two assignees have a cos-ang, of 1.0
print(assignee_names[assignee_names.pdpass==10030357])
print(assignee_names[assignee_names.pdpass==10030383])
# could these basically all be an array of zeros?!


                      cod    cod_fix    pdpass standard_name  uspto_assignee
123426  02 US corporation  No change  10030357      HGP CORP        250360.0
                                      cod    cod_fix    pdpass  \
93056  03 Foreign corp, incl. state-owned  No change  10030383   
93057  03 Foreign corp, incl. state-owned  No change  10030383   

                         standard_name  uspto_assignee  
93056   FA FOUQUET WERK FRAUZ & PLANCK        206080.0  
93057  FA FOUQUET WERKE FRAUZ & PLANCK        206080.0  


In [26]:
feature_vectors_df.head()

,icl_class,10030271.0,10156902.0,10112031.0,10030276.0,10172187.0,10679134.0,10030281.0,10577942.0,10036908.0,...,10030414.0,10140712.0,10286499.0,10679547.0,10699376.0,10078381.0,10116963.0,10235236.0,10198604.0,10966684.0
0,A41D,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A47D,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,B60S,0,0,0,2,0,0,0,0,0,...,0,0,1,1,6,0,0,0,0,0
3,E01F,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A47L,0,0,0,0,9,1,1,0,0,...,0,0,4,0,1,0,0,0,0,0


In [27]:
# get patents for specific assignee
x = pat76_06_assg_df_valid_pdpass[pat76_06_assg_df_valid_pdpass.pdpass ==10198901]
x.T

,1152,4981,9091,29506,31101,35205,39501,46936,50983,66720,...,971644,980581,1019729,1038984,1077822,1170881,1328326,1388927,2583892,2600602
allcites,3,16,4,2,8,1,2,0,0,1,...,2,1,7,0,18,11,1,1,0,0
appyear,1973,1973,1973,1974,1974,1974,1974,1975,1974,1974,...,1987,1985,1988,1987,1985,1990,1990,1992,1995,1993
asscode,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
assgnum,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
cat,3,3,3,3,3,1,1,3,3,3,...,1,3,3,3,3,3,3,1,3,3
cat_ocl,3,3,3,3,3,1,1,3,3,3,...,1,3,3,3,3,3,3,1,3,3
cclass,514/195,514/206,514/195,514/383,514/217,556/149,560/186,514/19,514/195,514/654,...,540/318,424/114,514/428,424/114,424/490,424/490,514/428,548/566,514/120,514/120
country,SW,SW,SW,SW,SW,SW,SW,SW,SW,SW,...,SE,SE,SE,SE,SE,SE,SE,SE,SE,SE
ddate,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2005-12-06 00:00:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT
gday,6,27,17,25,1,22,13,24,14,7,...,26,6,26,11,19,4,31,5,5,10


In [28]:
import networkx as nx
from networkx.algorithms import bipartite

In [29]:
feature_vectors_df.head()

,icl_class,10030271.0,10156902.0,10112031.0,10030276.0,10172187.0,10679134.0,10030281.0,10577942.0,10036908.0,...,10030414.0,10140712.0,10286499.0,10679547.0,10699376.0,10078381.0,10116963.0,10235236.0,10198604.0,10966684.0
0,A41D,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A47D,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,B60S,0,0,0,2,0,0,0,0,0,...,0,0,1,1,6,0,0,0,0,0
3,E01F,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A47L,0,0,0,0,9,1,1,0,0,...,0,0,4,0,1,0,0,0,0,0


In [30]:
new_index = feature_vectors_df.icl_class.values
feature_vectors_df_v2 = feature_vectors_df.reindex(new_index)
feature_vectors_df_v3 = feature_vectors_df_v2.drop('icl_class',axis=1)
feature_vectors_df_v3.head()

,10030271.0,10156902.0,10112031.0,10030276.0,10172187.0,10679134.0,10030281.0,10577942.0,10036908.0,10085072.0,...,10030414.0,10140712.0,10286499.0,10679547.0,10699376.0,10078381.0,10116963.0,10235236.0,10198604.0,10966684.0
A41D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A47D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B60S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E01F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A47L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
import scipy
# scipy.sparse.csr_matrix(df.values)
Z = scipy.sparse.csr_matrix(feature_vectors_df_v3.values)
Z

<3798x100 sparse matrix of type '<class 'numpy.float64'>'
	with 379800 stored elements in Compressed Sparse Row format>

In [32]:
B = nx.Graph()
B.add_nodes_from([1,2,3,4], bipartite=0) # Add the node attribute "bipartite"
B.add_nodes_from(['a','b','c'], bipartite=1)
B.add_edges_from([(1,'a'), (1,'b'), (2,'b'), (2,'c'), (3,'c'), (4,'a')])

In [33]:
XX = nx.bipartite.from_biadjacency_matrix(Z)

In [34]:
nx.bipartite.betweenness_centrality(XX,[1,2])

KeyboardInterrupt: 

In [ ]:
nx.draw(B)

In [ ]:
!sysctl hw.ncpu